In [0]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle

import xarray as xr
import rioxarray

import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import Resampling
from rasterio.vrt import WarpedVRT
from rasterio import shutil as rio_shutil

In [0]:
# paths
dirpath = '/Users/Sun-jooLee/Git_Projects/Myanmar_Survey_ML/data/geo'
# this is the reference grid - landscan (population)
refpath = f'{dirpath}/landscan/landscan-global-2017.tif'

In [0]:
# myanmar bounding box
min_lon = 92.3032344909 
min_lat = 9.93295990645 
max_lon = 101.180005324 
max_lat = 28.335945136 

### Reference grid vrt options

In [0]:
with rasterio.open(refpath) as src:
    # clip to myanmar
    win = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    win = Window(*[max(0, v) for v in win.flatten()])
    vrt_options = {
        "crs": src.crs.to_epsg(),
        "transform": src.window_transform(win),
        "width": int(win.width),
        "height": int(win.height),
    }
    
vrt_options.update({'resampling':Resampling.bilinear})
vrt_options

### Landscan (annual) - ref grid

In [0]:
# path
landscan_path = f'{dirpath}/landscan/landscan-global-2017.tif'
# open file
da = rioxarray.open_rasterio(landscan_path)
# clip to myanmar
lscn = da.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
# fill non value 
lscn = lscn.where(lscn != lscn.attrs['_FillValue'], np.nan)
# some cleaning
lscn = lscn.drop('band').drop('spatial_ref')
lscn.name = 'landscan'
lscn = lscn.to_dataset()
lscn = (
    lscn.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['landscan']})
    .rename({'x':'lon', 'y':'lat'})
    .squeeze('band')
)

In [0]:
# check 
lscn

In [0]:
# save
with open(f'{dirpath}/landscan/landscan2017_xarray.pickle', 'wb') as handle:
    pickle.dump(lscn, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Landcover (annual)

In [0]:
# path
landcover_path = f'netcdf:{dirpath}/landcover/C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1-002.nc:lccs_class'
# open transformed and clipped to myanmar
with rasterio.open(landcover_path) as src:
    with WarpedVRT(src, **vrt_options) as vrt:
        lcvr = rioxarray.open_rasterio(vrt)

In [0]:
# some cleaning - note, no non values in myanmar
lcvr = lcvr.drop('band').drop('spatial_ref')
lcvr.name = 'landcover'
lcvr = lcvr.to_dataset()
lcvr = (
    lcvr.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['landcover']})
    .rename({'x':'lon', 'y':'lat'})
    .squeeze('band')
)

In [0]:
# check
lcvr

In [0]:
# save
with open(f'{dirpath}/landcover/landcover2017_xarray.pickle', 'wb') as handle:
    pickle.dump(lcvr, handle, protocol=pickle.HIGHEST_PROTOCOL)

### VIIRS (monthly)

In [0]:
viirs_lst = [file for file in os.listdir(f'{dirpath}/viirs/2017/') if file!='.DS_Store']
viirs_xarrays = []

for file in viirs_lst:
    # path
    viirs_path = f'{dirpath}/viirs/2017/{file}'
    # open transformed and clipped to myanmar
    with rasterio.open(viirs_path) as src:
        with WarpedVRT(src, **vrt_options) as vrt:
            viirs = rioxarray.open_rasterio(vrt)
            
    # process one month dataset    
    yearmonth_tif = file.split('_')[-1]
    year = int(yearmonth_tif[:4])
    month = int(yearmonth_tif[4:6])
    
    # fill non value 
    viirs = viirs.where(viirs != viirs.attrs['_FillValue'], np.nan)
    # some cleaning
    viirs = viirs.drop('band').drop('spatial_ref')
    viirs.name = 'viirs'
    viirs = viirs.to_dataset()
    viirs = (
        viirs.expand_dims({'time':[datetime.date(year, month, 1)], 'source':['viirs']})
        .rename({'x':'lon', 'y':'lat'})
        .squeeze('band')
    )
    viirs_xarrays.append(viirs)

In [0]:
# check
print(len(viirs_xarrays))
viirs_all = xr.combine_by_coords(viirs_xarrays, combine_attrs='drop')
viirs_all

In [0]:
# save
with open(f'{dirpath}/viirs/2017/viirs2017_xarray.pickle', 'wb') as handle:
    pickle.dump(viirs_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

### FLDAS (monthly)

In [0]:
# all_vars = ['Evap_tavg','LWdown_f_tavg','Lwnet_tavg','Psurf_f_tavg','Qair_f_tavg','Qg_tavg','Qh_tavg','Qle_tavg','Qs_tavg','Qsb_tavg','RadT_tavg','Rainf_f_tavg','SWE_inst','SWdown_f_tavg','SnowCover_inst','SnowDepth_inst','Snowf_tavg','Swnet_tavg','Tair_f_tavg','Wind_f_tavg','SoilMoi00_10cm_tavg','SoilMoi10_40cm_tavg','SoilMoi40_100cm_tavg','SoilMoi100_200cm_tavg','SoilTemp00_10cm_tavg','SoilTemp10_40cm_tavg','SoilTemp40_100cm_tavg','SoilTemp100_200cm_tavg']
fldas_vars = ['Evap_tavg','Rainf_f_tavg','Tair_f_tavg','SoilMoi00_10cm_tavg','SoilMoi10_40cm_tavg','SoilMoi40_100cm_tavg','SoilMoi100_200cm_tavg','Qair_f_tavg','Qg_tavg','Qh_tavg','Qs_tavg']

In [0]:
fldas_lst = [file for file in os.listdir(f'{dirpath}/fldas/2017/') if file!='.DS_Store']
fldas_xarrays = []

for file in fldas_lst:
    for var in fldas_vars:
        # path
        fldas_path = f'netcdf:{dirpath}/fldas/2017/{file}:{var}'
        # open transformed and clipped to myanmar
        with rasterio.open(fldas_path) as src:
            with WarpedVRT(src, **vrt_options) as vrt:
                fldas = rioxarray.open_rasterio(vrt)

        # process one month dataset    
        yearmonth = file.split('.')[1]
        year = int(yearmonth[1:5])
        month = int(yearmonth[5:7])
        
        # fill non value 
        fldas = fldas.where(fldas != fldas.attrs['_FillValue'], np.nan)
        # some cleaning
        fldas = fldas.drop('band').drop('spatial_ref')
        fldas.name = var
        fldas = fldas.to_dataset()
        fldas = (
            fldas.expand_dims({'time':[datetime.date(year, month, 1)], 'source':['fldas']})
            .rename({'x':'lon', 'y':'lat'})
            .squeeze('band')
        )
        fldas_xarrays.append(fldas)

In [0]:
# check
print(len(fldas_xarrays))
fldas_all = xr.combine_by_coords(fldas_xarrays, combine_attrs='drop')
fldas_all

In [0]:
# save
with open(f'{dirpath}/fldas/2017/fldas2017_xarray.pickle', 'wb') as handle:
    pickle.dump(fldas_all, handle, protocol=pickle.HIGHEST_PROTOCOL)